In [78]:
import warnings
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import StandardScaler

warnings.simplefilter("ignore")

In [79]:
train = pd.read_csv("train.csv")

In [80]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [81]:
#suppression des colonnes non utiles
train = train.drop(['PassengerId'], axis=1)
train = train.drop(['Cabin'], axis=1)
train = train.drop(['Ticket'], axis=1)

In [82]:
#Remplacer les valeurs de la Q,S,C par 1, 2 et 3

Q = 0
S = 0
C = 0
N = 0

for i in range(len(train)):
    if train.iloc[i,8] == "Q":
        Q+=1
    elif train.iloc[i,8] == "S":
        S+=1
    elif train.iloc[i,8] == "C":
        C+=1
    else:
        N+=1
print("Q :", Q, " S :",S," C :",C," N :",N)



train['Embarked'] = train['Embarked'].replace(['Q'],'1')
train['Embarked'] = train['Embarked'].replace(['S'],'2')
train['Embarked'] = train['Embarked'].replace(['C'],'3')
train['Embarked'] = train['Embarked'].fillna('2')

Q : 77  S : 644  C : 168  N : 2


In [83]:
#Remplcaer les données catégoriques "male et female" par 1 et 0
train['Sex'] = train['Sex'].replace(['male'],'1')
train['Sex'] = train['Sex'].replace(['female'],'0')

In [84]:
#conversion colonne Name

newV = train['Name'].copy()
liste = []

for i in range(len(newV)):
    valeur = newV[i]
    chaine = re.findall(" ([A-Za-z]+)\.", valeur)[0]
    
    if chaine not in liste:
        liste.append(chaine)
    
    if chaine == "Mr":
        newV[i] = 1
    elif chaine == "Miss":
        newV[i] = 2
    elif chaine == "Mrs":
        newV[i] = 3
    elif chaine == "Master":
        newV[i] = 4
    elif chaine == "Dr":
        newV[i] = 5
    elif chaine == "Rev":
        newV[i] = 6
    elif chaine == "Don":
        newV[i] = 7
    elif chaine == "Mme":
        newV[i] = 8
    elif chaine == "Ms":
        newV[i] = 9
    elif chaine == "Major":
        newV[i] = 10
    elif chaine == "Lady":
        newV[i] = 11
    elif chaine == "Sir":
        newV[i] = 12
    elif chaine == "Mlle":
        newV[i] = 13
    elif chaine == "Col":
        newV[i] = 14
    elif chaine == "Capt":
        newV[i] = 15
    elif chaine == "Countess":
        newV[i] = 16
    elif chaine == "Jonkheer":
        newV[i] = 17
    else:
        newV[i] = 18
        

train['Name'] = newV 
print(liste)

['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms', 'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'Countess', 'Jonkheer']


In [85]:
averageAge = round(train['Age'].sum()/len(train))
train['Age'] = train['Age'].fillna(averageAge)

In [86]:
train['Embarked'] = train['Embarked'].astype('int64')
train['Name'] = train['Name'].astype('int64')
train['Sex'] = train['Sex'].astype('int64')

In [87]:
AvecValeursManquantes = len(train)
train = train.dropna()
SansValeursManquantes = len(train)

print("Avec Valeurs Manquantes :", AvecValeursManquantes,"  Sans Valeurs Manquantes :",SansValeursManquantes)

y = train['Survived']
train = train.drop(['Survived'], axis=1)
print(y)

Avec Valeurs Manquantes : 891   Sans Valeurs Manquantes : 891
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [88]:
import time
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
train = scaler.fit_transform(train)

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=0)


start_time = time.time()
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
prediction = mlp.predict(X_test)
print(accuracy_score(y_test, prediction))
print("--- %s seconds ---" % (time.time() - start_time))

0.8156424581005587
--- 0.6053063869476318 seconds ---


In [170]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential


In [171]:
train.shape

(891, 8)

In [172]:
#traduction de notre MLP avec le  scoore le plus haut sur KEras

model = Sequential()
model.add(Input(train.shape))
model.add(Dense(units=70, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))
model.compile(loss="BinaryCrossentropy", optimizer="adam", metrics=["accuracy"])

In [173]:
model.fit(X_train, y_train, batch_size=400, epochs = 10, verbose =1)
score = model.evaluate(X_test, y_test)

Epoch 1/10
2/2 [==============================] - 0s 1ms/step - loss: 0.6306 - accuracy: 0.6826
Epoch 2/10
2/2 [==============================] - 0s 935us/step - loss: 0.6191 - accuracy: 0.6966
Epoch 3/10
2/2 [==============================] - 0s 857us/step - loss: 0.6080 - accuracy: 0.7289
Epoch 4/10
2/2 [==============================] - 0s 917us/step - loss: 0.5975 - accuracy: 0.7444
Epoch 5/10
2/2 [==============================] - 0s 938us/step - loss: 0.5875 - accuracy: 0.7486
Epoch 6/10
2/2 [==============================] - 0s 839us/step - loss: 0.5779 - accuracy: 0.7528
Epoch 7/10
2/2 [==============================] - 0s 865us/step - loss: 0.5689 - accuracy: 0.7556
Epoch 8/10
2/2 [==============================] - 0s 893us/step - loss: 0.5603 - accuracy: 0.7640
Epoch 9/10
2/2 [==============================] - 0s 903us/step - loss: 0.5519 - accuracy: 0.7711
Epoch 10/10
6/6 [==============================] - 0s 541us/step - loss: 0.5194 - accuracy: 0.8212


In [174]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5193780660629272
Test accuracy: 0.8212290406227112


In [121]:
model.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 70)                630       
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 71        
Total params: 701
Trainable params: 701
Non-trainable params: 0
_________________________________________________________________
